In [13]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
import os 
from scipy.sparse import linalg
from pathlib import Path
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu" 

torch.set_default_dtype(torch.float64)
pi = torch.tensor(np.pi,dtype=torch.float64)

class model(nn.Module):
    """ ReLU k shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    k: degree of relu functions
    """
    def __init__(self, input_size, hidden_size1, num_classes,k = 1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
        self.k = k 
    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x))**self.k)
        return u1
    


def show_convergence_order(err_l2,exponent,dict_size, filename,write2file = False):
    
    if write2file:
        file_mode = "a" if os.path.exists(filename) else "w"
        f_write = open(filename, file_mode)
    
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
#     err_list2 = [err_h10[i] for i in neuron_nums ] 
    # f_write.write('M:{}, relu {} \n'.format(M,k))
    if write2file:
        f_write.write('dictionary size: {}\n'.format(dict_size))
        f_write.write("neuron num \t\t error \t\t order \t\t h10 error \\ order \n")
    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            # print(neuron_nums[i], end = "\t\t")
            # print(item, end = "\t\t")
            
            # print("*")
            print("{} \t\t {:.6f} \t\t *  \n".format(neuron_nums[i],item ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t * \t\t \n".format(neuron_nums[i],item ))
        else: 
            # print(neuron_nums[i], end = "\t\t")
            # print(item, end = "\t\t") 
            # print(np.log(err_list[i-1]/err_list[i])/np.log(2))
            print("{} \t\t {:.6f} \t\t {:.6f} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2) ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2) ))
    if write2file:     
        f_write.write("\n")
        f_write.close()

def show_convergence_order_latex(err_l2,exponent,k=1,d=1): 
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
#     err_list2 = [err_h10[i] for i in neuron_nums ] 
    # f_write.write('M:{}, relu {} \n'.format(M,k))
    # f_write.write('randomized dictionary size: {}\n'.format(N))
    # f_write.write("neuron num \t\t error \t\t order \t\t h10 error \\ order \n")
    l2_order = -1/2-(2*k + 1)/(2*d)
#     h10_order = -1/2-(2*(k-1) + 1)/(2*d)
#     print("neuron num  & \t $\|u-u_n \|_{L^2}$ & \t order $O(n^{{{}})$ & \t $ | u -u_n |_{H^1}$ & \t order $O(n^{{{}})$ \\\ \hline \hline ".format(l2_order,h10_order))
    print("neuron num  & \t $\\|u-u_n \\|_{{L^2}}$ & \t order $O(n^{{{:.2f}}})$  \\\\ \\hline \\hline ".format(l2_order))
    for i, item in enumerate(err_list):
        if i == 0: 
            # print(neuron_nums[i], end = "\t\t")
            # print(item, end = "\t\t")

            # print("*")
            print("{} \t\t & {:.6f} &\t\t *  \\\ \hline  \n".format(neuron_nums[i],item) )   
            # f_write.write("{} \t\t {} \t\t * \t\t {} \t\t * \n".format(neuron_nums[i],item, err_list2[i] ))
        else: 
            # print(neuron_nums[i], end = "\t\t")
            # print(item, end = "\t\t") 
            # print(np.log(err_list[i-1]/err_list[i])/np.log(2))
            print("{} \t\t &  {:.3e} &  \t\t {:.2f} \\\ \hline  \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2) ) )
            # f_write.write("{} \t\t {} \t\t {} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ))
    # f_write.write("\n")
    # f_write.close()



In [14]:
# def PiecewiseGQ3D(target, Nx, order): 
#     """
#     Parameters
#     ----------
#     target : 
#         Target function 
#     Nx: int 
#         number of intervals along the dimension. No Ny, assume Nx = Ny
#     order: int 
#         order of the Gauss Quadrature
#     """

#     # print("order: ",order )
#     x, w = np.polynomial.legendre.leggauss(order)
#     gauss_pts = np.array(np.meshgrid(x,x,x,indexing='ij')).reshape(3,-1).T
#     weight_list = np.array(np.meshgrid(w,w,w,indexing='ij'))
#     weights =   (weight_list[0]*weight_list[1]*weight_list[2]).ravel() 

#     gauss_pts =torch.tensor(gauss_pts)
#     weights = torch.tensor(weights)

#     h = 1/Nx # 100 intervals 
#     long_weights =  torch.tile(weights,(Nx**3,1))
#     long_weights = long_weights.reshape(-1,1)
#     long_weights = long_weights * h**3 /8 

#     integration_points = torch.tile(gauss_pts,(Nx**3,1))
#     scale_factor = h/2 
#     integration_points = scale_factor * integration_points

#     index = np.arange(1,Nx+1)-0.5
#     ordered_pairs = np.array(np.meshgrid(index,index,index,indexing='ij'))
#     ordered_pairs = ordered_pairs.reshape(3,-1).T

#     # print(ordered_pairs)
#     # print()
#     ordered_pairs = torch.tensor(ordered_pairs)
#     # print(ordered_pairs.size())
#     ordered_pairs = torch.tile(ordered_pairs, (1,order**3)) # number of GQ points
#     # print(ordered_pairs)

#     ordered_pairs =  ordered_pairs.reshape(-1,3)
#     # print(ordered_pairs)
#     translation = ordered_pairs*h 
#     # print(translation)

#     integration_points = integration_points + translation 
#     integration_points = integration_points.to(device)
#     # print(integration_points.size())
#     # func_values = integrand2_torch(integration_points)
#     func_values = target(integration_points)

#     long_weights = long_weights.to(device) 
#     integral_value = long_weights.T@func_values # this allows multiple functions, stored in columns
#     return integral_value


def PiecewiseGQ3D_weights_points(Nx, order): 
    """ A slight modification of PiecewiseGQ2D function that only needs the weights and integration points.
    Parameters
    ----------

    Nx: int 
        number of intervals along the dimension. No Ny, assume Nx = Ny
    order: int 
        order of the Gauss Quadrature

    Returns
    -------
    long_weights: torch.tensor
    integration_points: torch.tensor
    """

    """
    Parameters
    ----------
    target : 
        Target function 
    Nx: int 
        number of intervals along the dimension. No Ny, assume Nx = Ny
    order: int 
        order of the Gauss Quadrature
    """

    # print("order: ",order )
    x, w = np.polynomial.legendre.leggauss(order)
    gauss_pts = np.array(np.meshgrid(x,x,x,indexing='ij')).reshape(3,-1).T
    weight_list = np.array(np.meshgrid(w,w,w,indexing='ij'))
    weights =   (weight_list[0]*weight_list[1]*weight_list[2]).ravel() 

    gauss_pts =torch.tensor(gauss_pts)
    weights = torch.tensor(weights)

    h = 1/Nx # 100 intervals 
    long_weights =  torch.tile(weights,(Nx**3,1))
    long_weights = long_weights.reshape(-1,1)
    long_weights = long_weights * h**3 /8 

    integration_points = torch.tile(gauss_pts,(Nx**3,1))
    # print("shape of integration_points", integration_points.size())
    scale_factor = h/2 
    integration_points = scale_factor * integration_points

    index = np.arange(1,Nx+1)-0.5
    ordered_pairs = np.array(np.meshgrid(index,index,index,indexing='ij'))
    ordered_pairs = ordered_pairs.reshape(3,-1).T

    # print(ordered_pairs)
    # print()
    ordered_pairs = torch.tensor(ordered_pairs)
    # print(ordered_pairs.size())
    ordered_pairs = torch.tile(ordered_pairs, (1,order**3)) # number of GQ points
    # print(ordered_pairs)

    ordered_pairs =  ordered_pairs.reshape(-1,3)
    # print(ordered_pairs)
    translation = ordered_pairs*h 
    # print(translation)

    integration_points = integration_points + translation 

    return long_weights, integration_points



def minimize_linear_layer_explicit_assemble(model,target,weights, integration_points,solver="direct"):
    """
    calls the following functions (dependency): 
    1. GQ_piecewise_2D
    input: the nn model containing parameter 
    1. define the loss function  
    2. take derivative to extract the linear system A
    3. call the cg solver in scipy to solve the linear system 
    output: sol. solution of Ax = b
    """
    start_time = time.time() 
    w = model.fc1.weight.data 
    b = model.fc1.bias.data 
    basis_value_col = F.relu(integration_points @ w.t()+ b)**(model.k) 
    weighted_basis_value_col = basis_value_col * weights 
    jac = weighted_basis_value_col.t() @ basis_value_col 
     
    rhs = weighted_basis_value_col.t() @ (target(integration_points)) 
    print("assembling the matrix time taken: ", time.time()-start_time) 
    start_time = time.time()    
    if solver == "cg": 
        sol, exit_code = linalg.cg(np.array(jac.detach().cpu()),np.array(rhs.detach().cpu()),tol=1e-12)
        sol = torch.tensor(sol).view(1,-1)
    elif solver == "direct": 
#         sol = np.linalg.inv( np.array(jac.detach().cpu()) )@np.array(rhs.detach().cpu())
        sol = (torch.linalg.solve( jac.detach(), rhs.detach())).view(1,-1)
    elif solver == "ls":
        sol = (torch.linalg.lstsq(jac.detach().cpu(),rhs.detach().cpu(),driver='gelsd').solution).view(1,-1)
        # sol = (torch.linalg.lstsq(jac.detach(),rhs.detach()).solution).view(1,-1) # gpu/cpu, driver = 'gels', cannot solve singular
    print("solving Ax = b time taken: ", time.time()-start_time)
    return sol 


## 3D QMC greedy algorithm 


In [15]:
def generate_relu_dict3D(N_list):
    N1 = N_list[0]
    N2 = N_list[1]
    N3 = N_list[2]
    
    N = N1*N2*N3 
    theta1 = np.linspace(0, pi, N1, endpoint= True).reshape(N1,1)
    theta2 = np.linspace(0, 2*pi, N2, endpoint= False).reshape(N2,1)
    b = np.linspace(-1.732, 1.732, N3,endpoint=False).reshape(N3,1) # threshold: 3**0.5  
    coord3 = np.array(np.meshgrid(theta1,theta2,b,indexing='ij'))
    coord3 = coord3.reshape(3,-1).T # N1*N2*N3 x 3. coordinates for the grid points 
    coord3 = torch.tensor(coord3) 

    f1 = torch.zeros(N,1) 
    f2 = torch.zeros(N,1)
    f3 = torch.zeros(N,1)
    f4 = torch.zeros(N,1)

    f1[:,0] = torch.cos(coord3[:,0]) 
    f2[:,0] = torch.sin(coord3[:,0]) * torch.cos(coord3[:,1])
    f3[:,0] = torch.sin(coord3[:,0]) * torch.sin(coord3[:,1])
    f4[:,0] = coord3[:,2] 

    Wb_tensor = torch.cat([f1,f2,f3,f4],1) # N x 4 
    return Wb_tensor

def generate_relu_dict3D_QMC(s,N0):
    #Sob = torch.quasirandom.SobolEngine(dimension =3, scramble= True, seed=None) 
    #samples = Sob.draw(N0).double() 

    # Monte Carlo 
    samples = torch.rand(s*N0,3) 
    T =torch.tensor([[pi,0,0],[0,2*pi,0],[0,0,1.732*2]])
    shift = torch.tensor([0,0,-1.732])
    samples = samples@T + shift 

    f1 = torch.zeros(s*N0,1) 
    f2 = torch.zeros(s*N0,1)
    f3 = torch.zeros(s*N0,1)
    f4 = torch.zeros(s*N0,1)

    f1[:,0] = torch.cos(samples[:,0]) 
    f2[:,0] = torch.sin(samples[:,0]) * torch.cos(samples[:,1])
    f3[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1])
    f4[:,0] = samples[:,2] 

    Wb_tensor = torch.cat([f1,f2,f3,f4],1) # N x 4 
    return Wb_tensor

def MonteCarlo_Sobol_dDim_weights_points(M ,d = 4):
    Sob_integral = torch.quasirandom.SobolEngine(dimension =d, scramble= False, seed=None) 
    integration_points = Sob_integral.draw(M).double() 
    integration_points = integration_points.to(device)
    weights = torch.ones(M,1).to(device)/M 
    return weights.to(device), integration_points.to(device) 


In [16]:
def OGAL2FittingReLU3D(my_model,target,N_list,num_epochs,plot_freq, Nx, order, k =1, linear_solver = "direct"): 

    """ Orthogonal greedy algorithm using 1D ReLU dictionary over [-pi,pi]
    Parameters
    ----------
    my_model: 
        nn model 
    target: 
        target function
    num_epochs: int 
        number of training epochs 
    integration_intervals: int 
        number of subintervals for piecewise numerical quadrature 

    Returns
    -------
    err: tensor 
        rank 1 torch tensor to record the L2 error history  
    model: 
        trained nn model 
    """
    #Todo Done
    gw_expand, integration_points = PiecewiseGQ3D_weights_points(Nx, order)
    gw_expand = gw_expand.to(device)
    integration_points = integration_points.to(device)

    err = torch.zeros(num_epochs+1)
    if my_model == None: 
        func_values = target(integration_points)
        num_neuron = 0

        list_b = []
        list_w = []

    else: 
        func_values = target(integration_points) - my_model(integration_points).detach()
        bias = my_model.fc1.bias.detach().data
        weights = my_model.fc1.weight.detach().data
        num_neuron = int(bias.size(0))

        list_b = list(bias)
        list_w = list(weights)
    
    # initial error Todo Done
    func_values_sqrd = func_values*func_values

    err[0]= torch.sum(func_values_sqrd*gw_expand)**0.5
    start_time = time.time()
    
    solver = linear_solver
    print("using linear solver: ",solver)
#     relu_dict_parameters = generate_relu_dict3D_QMC(s,N0).to(device) 
    N = np.prod(N_list) 
    relu_dict_parameters = generate_relu_dict3D(N_list).to(device)
    
    for i in range(num_epochs): 
#         relu_dict_parameters = generate_relu_dict3D_QMC(s,N0).to(device) 
        print("epoch: ",i+1, end = '\t')
        if num_neuron == 0: 
            func_values = target(integration_points)
        else: 
            func_values = target(integration_points) - my_model(integration_points).detach()

        weight_func_values = func_values*gw_expand  
        basis_values = (F.relu( torch.matmul(integration_points,relu_dict_parameters[:,0:3].T ) - relu_dict_parameters[:,3])**k).T # uses broadcasting

        output = torch.abs(torch.matmul(basis_values,weight_func_values)) # 
        neuron_index = torch.argmax(output.flatten())
        
        # print(neuron_index)
        list_w.append(relu_dict_parameters[neuron_index,0:3]) # 
        list_b.append(-relu_dict_parameters[neuron_index,3])
        num_neuron += 1
        my_model = model(3,num_neuron,1,k).to(device)
        w_tensor = torch.stack(list_w, 0 ) 
        b_tensor = torch.tensor(list_b)
        my_model.fc1.weight.data[:,:] = w_tensor[:,:]
        my_model.fc1.bias.data[:] = b_tensor[:]

#         sol = minimize_linear_layer(my_model,target,solver,Nx,order)
        sol = minimize_linear_layer_explicit_assemble(my_model,target,gw_expand,integration_points,solver)

        my_model.fc2.weight.data[0,:] = sol[:]
        # if (i+1)%plot_freq == 0: 
        #     plot_2D(my_model.cpu())
        #     my_model = my_model.to(device)

        func_values = target(integration_points) - my_model(integration_points).detach()
        func_values_sqrd = func_values*func_values

        #Todo Done 
        err[i+1]= torch.sum(func_values_sqrd*gw_expand)**0.5
    print("time taken: ",time.time() - start_time)
    return err, my_model


def OGAL2FittingReLU3D_QMC(my_model,target,s,N0,num_epochs,plot_freq, Nx, order, k =1, linear_solver = "direct"): 

    """ Orthogonal greedy algorithm on (0,1)^3 
    """
    #Todo Done
    gw_expand, integration_points = PiecewiseGQ3D_weights_points(Nx, order)
    gw_expand = gw_expand.to(device)
    integration_points = integration_points.to(device)

    err = torch.zeros(num_epochs+1)
    if my_model == None: 
        func_values = target(integration_points)
        num_neuron = 0

        list_b = []
        list_w = []

    else: 
        func_values = target(integration_points) - my_model(integration_points).detach()
        bias = my_model.fc1.bias.detach().data
        weights = my_model.fc1.weight.detach().data
        num_neuron = int(bias.size(0))

        list_b = list(bias)
        list_w = list(weights)
    


    func_values_sqrd = func_values*func_values
    # print(func_values_sqrd.size())
    # print(gw_expand.size() ) 

    err[0]= torch.sum(func_values_sqrd*gw_expand)**0.5
    start_time = time.time()
    
    solver = linear_solver
    print("using linear solver: ",solver)
    for i in range(num_epochs): 
        relu_dict_parameters = generate_relu_dict3D_QMC(s,N0).to(device) 
        print("epoch: ",i+1, end = '\t')
        if num_neuron == 0: 
            func_values = target(integration_points)
        else: 
            func_values = target(integration_points) - my_model(integration_points).detach()

        weight_func_values = func_values*gw_expand  
        basis_values = (F.relu( torch.matmul(integration_points,relu_dict_parameters[:,0:3].T ) - relu_dict_parameters[:,3])**k).T # uses broadcasting

        output = torch.abs(torch.matmul(basis_values,weight_func_values)) # 
        neuron_index = torch.argmax(output.flatten())
        
        # print(neuron_index)
        list_w.append(relu_dict_parameters[neuron_index,0:3]) # 
        list_b.append(-relu_dict_parameters[neuron_index,3])
        num_neuron += 1
        my_model = model(3,num_neuron,1,k).to(device)
        w_tensor = torch.stack(list_w, 0 ) 
        b_tensor = torch.tensor(list_b)
        my_model.fc1.weight.data[:,:] = w_tensor[:,:]
        my_model.fc1.bias.data[:] = b_tensor[:]

        sol = minimize_linear_layer_explicit_assemble(my_model,target,gw_expand,integration_points,solver)

        my_model.fc2.weight.data[0,:] = sol[:]
        func_values = target(integration_points) - my_model(integration_points).detach()
        func_values_sqrd = func_values*func_values

        #Todo Done 
        err[i+1]= torch.sum(func_values_sqrd*gw_expand)**0.5
    print("time taken: ",time.time() - start_time)
    return err, my_model





## Randomized dictionaries

In [17]:
def target(x):
    return torch.sin(pi*x[:,0:1])*torch.sin(pi*x[:,1:2])*torch.sin(pi*x[:,2:3]) 

function_name = "sinpipipi" 
filename_write = "data/3DQMCOGA-{}-order.txt".format(function_name)
dim = 3 
Nx = 25   
order = 3    
save = True
write2file = True

f_write = open(filename_write, "a")
f_write.write("Integration points: Nx {}, order {} \n".format(Nx,order))
f_write.close() 

for N0 in [2*1,2**2,2**3,2**4,2**5]: 
    f_write = open(filename_write, "a")
    my_model = None 
    s = 2**5
    N = 2 * N0 
    relu_k = 1 
    exponent = 9
    num_epochs = 2**exponent  
    plot_freq = 1 
    err_QMC2, my_model = OGAL2FittingReLU3D_QMC(my_model,target,s,N0,num_epochs,plot_freq, Nx, order, k =relu_k, linear_solver = "ls")
    
    if save: 
        folder = 'data/'
        filename = folder + 'err_OGA_3D_{}_neuron_{}_N_{}_randomized.pt'.format(function_name,num_epochs,s*N0)
        torch.save(err_QMC2,filename) 
        folder = 'data/'
        filename = folder + 'model_OGA_3D_{}_neuron_{}_N_{}_randomized.pt'.format(function_name,num_epochs,s*N0)
        torch.save(my_model,filename)
        
    show_convergence_order(err_QMC2,exponent,N,filename_write,write2file = write2file)
    show_convergence_order_latex(err_QMC2,exponent,k=relu_k,d=dim)
    


using linear solver:  ls
epoch:  1	assembling the matrix time taken:  0.00024199485778808594
solving Ax = b time taken:  0.00015807151794433594
epoch:  2	assembling the matrix time taken:  0.00025081634521484375
solving Ax = b time taken:  0.006290912628173828
epoch:  3	assembling the matrix time taken:  0.00022912025451660156
solving Ax = b time taken:  0.0004143714904785156
epoch:  4	assembling the matrix time taken:  0.00022935867309570312
solving Ax = b time taken:  0.0004324913024902344
epoch:  5	assembling the matrix time taken:  0.00023627281188964844
solving Ax = b time taken:  0.00045371055603027344
epoch:  6	assembling the matrix time taken:  0.00024199485778808594
solving Ax = b time taken:  0.00048279762268066406
epoch:  7	assembling the matrix time taken:  0.0002346038818359375
solving Ax = b time taken:  0.0005125999450683594
epoch:  8	assembling the matrix time taken:  0.0002415180206298828
solving Ax = b time taken:  0.0005195140838623047
epoch:  9	assembling the matrix

solving Ax = b time taken:  0.0049228668212890625
epoch:  84	assembling the matrix time taken:  0.00023436546325683594
solving Ax = b time taken:  0.0033271312713623047
epoch:  85	assembling the matrix time taken:  0.00023365020751953125
solving Ax = b time taken:  0.0034341812133789062
epoch:  86	assembling the matrix time taken:  0.00023484230041503906
solving Ax = b time taken:  0.0034110546112060547
epoch:  87	assembling the matrix time taken:  0.000232696533203125
solving Ax = b time taken:  0.0034859180450439453
epoch:  88	assembling the matrix time taken:  0.00023412704467773438
solving Ax = b time taken:  0.0033774375915527344
epoch:  89	assembling the matrix time taken:  0.00023651123046875
solving Ax = b time taken:  0.0035314559936523438
epoch:  90	assembling the matrix time taken:  0.00024390220642089844
solving Ax = b time taken:  0.0035562515258789062
epoch:  91	assembling the matrix time taken:  0.00023365020751953125
solving Ax = b time taken:  0.0036439895629882812
epo

solving Ax = b time taken:  0.008814334869384766
epoch:  160	assembling the matrix time taken:  0.0002491474151611328
solving Ax = b time taken:  0.007508754730224609
epoch:  161	assembling the matrix time taken:  0.00023245811462402344
solving Ax = b time taken:  0.007757663726806641
epoch:  162	assembling the matrix time taken:  0.0002434253692626953
solving Ax = b time taken:  0.007684469223022461
epoch:  163	assembling the matrix time taken:  0.00023627281188964844
solving Ax = b time taken:  0.00785207748413086
epoch:  164	assembling the matrix time taken:  0.0002377033233642578
solving Ax = b time taken:  0.007742881774902344
epoch:  165	assembling the matrix time taken:  0.00023674964904785156
solving Ax = b time taken:  0.00795125961303711
epoch:  166	assembling the matrix time taken:  0.0002455711364746094
solving Ax = b time taken:  0.007917642593383789
epoch:  167	assembling the matrix time taken:  0.00024700164794921875
solving Ax = b time taken:  0.008075952529907227
epoch

solving Ax = b time taken:  0.014049053192138672
epoch:  232	assembling the matrix time taken:  0.00026988983154296875
solving Ax = b time taken:  0.013484477996826172
epoch:  233	assembling the matrix time taken:  0.0002944469451904297
solving Ax = b time taken:  0.014265298843383789
epoch:  234	assembling the matrix time taken:  0.0002543926239013672
solving Ax = b time taken:  0.014226675033569336
epoch:  235	assembling the matrix time taken:  0.000263214111328125
solving Ax = b time taken:  0.014319896697998047
epoch:  236	assembling the matrix time taken:  0.0002503395080566406
solving Ax = b time taken:  0.01420283317565918
epoch:  237	assembling the matrix time taken:  0.00025582313537597656
solving Ax = b time taken:  0.01454925537109375
epoch:  238	assembling the matrix time taken:  0.0002484321594238281
solving Ax = b time taken:  0.01446390151977539
epoch:  239	assembling the matrix time taken:  0.0002765655517578125
solving Ax = b time taken:  0.014650583267211914
epoch:  2

epoch:  303	assembling the matrix time taken:  0.0002720355987548828
solving Ax = b time taken:  0.02129340171813965
epoch:  304	assembling the matrix time taken:  0.00025343894958496094
solving Ax = b time taken:  0.020625591278076172
epoch:  305	assembling the matrix time taken:  0.00028061866760253906
solving Ax = b time taken:  0.021354198455810547
epoch:  306	assembling the matrix time taken:  0.0002646446228027344
solving Ax = b time taken:  0.02112603187561035
epoch:  307	assembling the matrix time taken:  0.0002598762512207031
solving Ax = b time taken:  0.021564006805419922
epoch:  308	assembling the matrix time taken:  0.0002689361572265625
solving Ax = b time taken:  0.021311283111572266
epoch:  309	assembling the matrix time taken:  0.0002570152282714844
solving Ax = b time taken:  0.02155470848083496
epoch:  310	assembling the matrix time taken:  0.00026679039001464844
solving Ax = b time taken:  0.021575212478637695
epoch:  311	assembling the matrix time taken:  0.0002665

...

solving Ax = b time taken:  0.013457536697387695
epoch:  212	assembling the matrix time taken:  0.0002512931823730469
solving Ax = b time taken:  0.011436700820922852
epoch:  213	assembling the matrix time taken:  0.0002315044403076172
solving Ax = b time taken:  0.013420581817626953
epoch:  214	assembling the matrix time taken:  0.00024390220642089844
solving Ax = b time taken:  0.011807918548583984
epoch:  215	assembling the matrix time taken:  0.000232696533203125
solving Ax = b time taken:  0.01364445686340332
epoch:  216	assembling the matrix time taken:  0.0002491474151611328
solving Ax = b time taken:  0.011678457260131836
epoch:  217	assembling the matrix time taken:  0.0002334117889404297
solving Ax = b time taken:  0.013932943344116211
epoch:  218	assembling the matrix time taken:  0.0002453327178955078
solving Ax = b time taken:  0.012191534042358398
epoch:  219	assembling the matrix time taken:  0.00023126602172851562
solving Ax = b time taken:  0.014034271240234375
epoch: 

solving Ax = b time taken:  0.022742509841918945
epoch:  284	assembling the matrix time taken:  0.00025200843811035156
solving Ax = b time taken:  0.018120765686035156
epoch:  285	assembling the matrix time taken:  0.000232696533203125
solving Ax = b time taken:  0.02198481559753418
epoch:  286	assembling the matrix time taken:  0.00024271011352539062
solving Ax = b time taken:  0.018729686737060547
epoch:  287	assembling the matrix time taken:  0.00023055076599121094
solving Ax = b time taken:  0.022050142288208008
epoch:  288	assembling the matrix time taken:  0.00024366378784179688
solving Ax = b time taken:  0.018451452255249023
epoch:  289	assembling the matrix time taken:  0.0002334117889404297
solving Ax = b time taken:  0.01941084861755371
epoch:  290	assembling the matrix time taken:  0.00024700164794921875
solving Ax = b time taken:  0.019460678100585938
epoch:  291	assembling the matrix time taken:  0.00023245811462402344
solving Ax = b time taken:  0.020345449447631836
epoc

solving Ax = b time taken:  0.02905750274658203
epoch:  354	assembling the matrix time taken:  0.0002532005310058594
solving Ax = b time taken:  0.026868343353271484
epoch:  355	assembling the matrix time taken:  0.00023412704467773438
solving Ax = b time taken:  0.028331518173217773
epoch:  356	assembling the matrix time taken:  0.00023174285888671875
solving Ax = b time taken:  0.027779817581176758
epoch:  357	assembling the matrix time taken:  0.00023412704467773438
solving Ax = b time taken:  0.02845001220703125
epoch:  358	assembling the matrix time taken:  0.000232696533203125
solving Ax = b time taken:  0.02838277816772461
epoch:  359	assembling the matrix time taken:  0.0002319812774658203
solving Ax = b time taken:  0.02886343002319336
epoch:  360	assembling the matrix time taken:  0.0002751350402832031
solving Ax = b time taken:  0.028036832809448242
epoch:  361	assembling the matrix time taken:  0.00023889541625976562
solving Ax = b time taken:  0.02891373634338379
epoch:  3

epoch:  425	assembling the matrix time taken:  0.00023651123046875
solving Ax = b time taken:  0.0364382266998291
epoch:  426	assembling the matrix time taken:  0.0002353191375732422
solving Ax = b time taken:  0.03679156303405762
epoch:  427	assembling the matrix time taken:  0.00024390220642089844
solving Ax = b time taken:  0.03637266159057617
epoch:  428	assembling the matrix time taken:  0.00024580955505371094
solving Ax = b time taken:  0.03653216361999512
epoch:  429	assembling the matrix time taken:  0.00023317337036132812
solving Ax = b time taken:  0.03670024871826172
epoch:  430	assembling the matrix time taken:  0.0002315044403076172
solving Ax = b time taken:  0.03769993782043457
epoch:  431	assembling the matrix time taken:  0.00024390220642089844
solving Ax = b time taken:  0.036920785903930664
epoch:  432	assembling the matrix time taken:  0.00023317337036132812
solving Ax = b time taken:  0.03637576103210449
epoch:  433	assembling the matrix time taken:  0.000235557556

epoch:  497	assembling the matrix time taken:  0.0002353191375732422
solving Ax = b time taken:  0.047252655029296875
epoch:  498	assembling the matrix time taken:  0.00023293495178222656
solving Ax = b time taken:  0.04729938507080078
epoch:  499	assembling the matrix time taken:  0.00023484230041503906
solving Ax = b time taken:  0.04859805107116699
epoch:  500	assembling the matrix time taken:  0.00023484230041503906
solving Ax = b time taken:  0.04661059379577637
epoch:  501	assembling the matrix time taken:  0.00023221969604492188
solving Ax = b time taken:  0.04804253578186035
epoch:  502	assembling the matrix time taken:  0.00023436546325683594
solving Ax = b time taken:  0.047849416732788086
epoch:  503	assembling the matrix time taken:  0.00023436546325683594
solving Ax = b time taken:  0.04861044883728027
epoch:  504	assembling the matrix time taken:  0.00023412704467773438
solving Ax = b time taken:  0.047765254974365234
epoch:  505	assembling the matrix time taken:  0.00023

## Deterministic dictionary

In [18]:
def target(x):
    return torch.sin(pi*x[:,0:1])*torch.sin(pi*x[:,1:2])*torch.sin(pi*x[:,2:3]) 



function_name = "sinpipipi" 
filename_write = "data/3DOGA-{}-order.txt".format(function_name)
dim = 3 
Nx = 25   
order = 3    
save = True
write2file = True

f_write = open(filename_write, "a")
f_write.write("Integration points: Nx {}, order {} \n".format(Nx,order))
f_write.close() 

for N_list in [[2**4,2**4,2**4],[2**4,2**4,2**3],[2**4,2**3,2**3]]: 
    save = True 
    f_write = open(filename_write, "a")
    my_model = None 
    relu_k = 1 
    exponent = 9  
    num_epochs = 2**exponent  
    plot_freq = 1 
    N = np.prod(N_list)
    err_QMC2, my_model = OGAL2FittingReLU3D(my_model,target,N_list,num_epochs,plot_freq, Nx, order, k =relu_k, linear_solver = "ls")
    
    if save: 
        folder = 'data/'
        filename = folder + 'err_OGA_3D_{}_neuron_{}_N_{}_deterministic.pt'.format(function_name,num_epochs,N)
        torch.save(err_QMC2,filename) 
        folder = 'data/'
        filename = folder + 'model_OGA_3D_{}_neuron_{}_N_{}_deterministic.pt'.format(function_name,num_epochs,N)
        torch.save(my_model,filename)
            
    show_convergence_order(err_QMC2,exponent,N,filename_write,write2file = write2file)
    show_convergence_order_latex(err_QMC2,exponent,k=relu_k,d=dim)
    
    

using linear solver:  ls
epoch:  1	assembling the matrix time taken:  0.0002460479736328125
solving Ax = b time taken:  0.00016117095947265625
epoch:  2	assembling the matrix time taken:  0.0002658367156982422
solving Ax = b time taken:  0.0061838626861572266
epoch:  3	assembling the matrix time taken:  0.00023055076599121094
solving Ax = b time taken:  0.0002067089080810547
epoch:  4	assembling the matrix time taken:  0.00023031234741210938
solving Ax = b time taken:  0.00020742416381835938
epoch:  5	assembling the matrix time taken:  0.00024318695068359375
solving Ax = b time taken:  0.00022292137145996094
epoch:  6	assembling the matrix time taken:  0.0002391338348388672
solving Ax = b time taken:  0.00024199485778808594
epoch:  7	assembling the matrix time taken:  0.0002377033233642578
solving Ax = b time taken:  0.0002930164337158203
epoch:  8	assembling the matrix time taken:  0.00023865699768066406
solving Ax = b time taken:  0.0003120899200439453
epoch:  9	assembling the matrix

epoch:  72	assembling the matrix time taken:  0.00024056434631347656
solving Ax = b time taken:  0.002794504165649414
epoch:  73	assembling the matrix time taken:  0.00023984909057617188
solving Ax = b time taken:  0.002937793731689453
epoch:  74	assembling the matrix time taken:  0.0002384185791015625
solving Ax = b time taken:  0.0029671192169189453
epoch:  75	assembling the matrix time taken:  0.00024271011352539062
solving Ax = b time taken:  0.003025531768798828
epoch:  76	assembling the matrix time taken:  0.0002474784851074219
solving Ax = b time taken:  0.0030307769775390625
epoch:  77	assembling the matrix time taken:  0.0002474784851074219
solving Ax = b time taken:  0.0031554698944091797
epoch:  78	assembling the matrix time taken:  0.000240325927734375
solving Ax = b time taken:  0.0031387805938720703
epoch:  79	assembling the matrix time taken:  0.000240325927734375
solving Ax = b time taken:  0.0032367706298828125
epoch:  80	assembling the matrix time taken:  0.0002467632

epoch:  144	assembling the matrix time taken:  0.0002396106719970703
solving Ax = b time taken:  0.006635427474975586
epoch:  145	assembling the matrix time taken:  0.0002410411834716797
solving Ax = b time taken:  0.007896900177001953
epoch:  146	assembling the matrix time taken:  0.00025010108947753906
solving Ax = b time taken:  0.007876157760620117
epoch:  147	assembling the matrix time taken:  0.0002467632293701172
solving Ax = b time taken:  0.00802469253540039
epoch:  148	assembling the matrix time taken:  0.0002422332763671875
solving Ax = b time taken:  0.007822990417480469
epoch:  149	assembling the matrix time taken:  0.00024127960205078125
solving Ax = b time taken:  0.008114099502563477
epoch:  150	assembling the matrix time taken:  0.0002415180206298828
solving Ax = b time taken:  0.008025169372558594
epoch:  151	assembling the matrix time taken:  0.00023937225341796875
solving Ax = b time taken:  0.008156299591064453
epoch:  152	assembling the matrix time taken:  0.00024

epoch:  215	assembling the matrix time taken:  0.0002372264862060547
solving Ax = b time taken:  0.013549327850341797
epoch:  216	assembling the matrix time taken:  0.0002465248107910156
solving Ax = b time taken:  0.011644840240478516
epoch:  217	assembling the matrix time taken:  0.0002346038818359375
solving Ax = b time taken:  0.013897895812988281
epoch:  218	assembling the matrix time taken:  0.000247955322265625
solving Ax = b time taken:  0.012238502502441406
epoch:  219	assembling the matrix time taken:  0.00023603439331054688
solving Ax = b time taken:  0.014163732528686523
epoch:  220	assembling the matrix time taken:  0.00025582313537597656
solving Ax = b time taken:  0.012132406234741211
epoch:  221	assembling the matrix time taken:  0.0002377033233642578
solving Ax = b time taken:  0.014152765274047852
epoch:  222	assembling the matrix time taken:  0.00024771690368652344
solving Ax = b time taken:  0.012628793716430664
epoch:  223	assembling the matrix time taken:  0.00023

epoch:  285	assembling the matrix time taken:  0.00024771690368652344
solving Ax = b time taken:  0.022031068801879883
epoch:  286	assembling the matrix time taken:  0.000244140625
solving Ax = b time taken:  0.018729686737060547
epoch:  287	assembling the matrix time taken:  0.00023555755615234375
solving Ax = b time taken:  0.02212214469909668
epoch:  288	assembling the matrix time taken:  0.0002455711364746094
solving Ax = b time taken:  0.01848006248474121
epoch:  289	assembling the matrix time taken:  0.00023865699768066406
solving Ax = b time taken:  0.019440174102783203
epoch:  290	assembling the matrix time taken:  0.00024437904357910156
solving Ax = b time taken:  0.019461870193481445
epoch:  291	assembling the matrix time taken:  0.00023674964904785156
solving Ax = b time taken:  0.01961040496826172
epoch:  292	assembling the matrix time taken:  0.00026297569274902344
solving Ax = b time taken:  0.019368886947631836
epoch:  293	assembling the matrix time taken:  0.00024294853

...

epoch:  172	assembling the matrix time taken:  0.0002391338348388672
solving Ax = b time taken:  0.008125543594360352
epoch:  173	assembling the matrix time taken:  0.00023412704467773438
solving Ax = b time taken:  0.008350133895874023
epoch:  174	assembling the matrix time taken:  0.0002334117889404297
solving Ax = b time taken:  0.008304595947265625
epoch:  175	assembling the matrix time taken:  0.0002334117889404297
solving Ax = b time taken:  0.008514642715454102
epoch:  176	assembling the matrix time taken:  0.00023436546325683594
solving Ax = b time taken:  0.008296966552734375
epoch:  177	assembling the matrix time taken:  0.0002346038818359375
solving Ax = b time taken:  0.008658885955810547
epoch:  178	assembling the matrix time taken:  0.0002353191375732422
solving Ax = b time taken:  0.008588552474975586
epoch:  179	assembling the matrix time taken:  0.0002372264862060547
solving Ax = b time taken:  0.008778572082519531
epoch:  180	assembling the matrix time taken:  0.00023

epoch:  244	assembling the matrix time taken:  0.00023674964904785156
solving Ax = b time taken:  0.013614416122436523
epoch:  245	assembling the matrix time taken:  0.00023555755615234375
solving Ax = b time taken:  0.013894796371459961
epoch:  246	assembling the matrix time taken:  0.00024247169494628906
solving Ax = b time taken:  0.013868093490600586
epoch:  247	assembling the matrix time taken:  0.00023603439331054688
solving Ax = b time taken:  0.013976335525512695
epoch:  248	assembling the matrix time taken:  0.0002353191375732422
solving Ax = b time taken:  0.013702154159545898
epoch:  249	assembling the matrix time taken:  0.00023889541625976562
solving Ax = b time taken:  0.01414036750793457
epoch:  250	assembling the matrix time taken:  0.00023317337036132812
solving Ax = b time taken:  0.013985395431518555
epoch:  251	assembling the matrix time taken:  0.0002346038818359375
solving Ax = b time taken:  0.014217615127563477
epoch:  252	assembling the matrix time taken:  0.00

epoch:  315	assembling the matrix time taken:  0.00024247169494628906
solving Ax = b time taken:  0.01900196075439453
epoch:  316	assembling the matrix time taken:  0.0002338886260986328
solving Ax = b time taken:  0.0187835693359375
epoch:  317	assembling the matrix time taken:  0.00023317337036132812
solving Ax = b time taken:  0.019126415252685547
epoch:  318	assembling the matrix time taken:  0.0002353191375732422
solving Ax = b time taken:  0.019005537033081055
epoch:  319	assembling the matrix time taken:  0.00024390220642089844
solving Ax = b time taken:  0.019345760345458984
epoch:  320	assembling the matrix time taken:  0.00023484230041503906
solving Ax = b time taken:  0.01912665367126465
epoch:  321	assembling the matrix time taken:  0.0002353191375732422
solving Ax = b time taken:  0.022742033004760742
epoch:  322	assembling the matrix time taken:  0.0002467632293701172
solving Ax = b time taken:  0.021048545837402344
epoch:  323	assembling the matrix time taken:  0.0002431

solving Ax = b time taken:  0.027936935424804688
epoch:  386	assembling the matrix time taken:  0.00023651123046875
solving Ax = b time taken:  0.02902078628540039
epoch:  387	assembling the matrix time taken:  0.00023627281188964844
solving Ax = b time taken:  0.028120040893554688
epoch:  388	assembling the matrix time taken:  0.0002357959747314453
solving Ax = b time taken:  0.028905391693115234
epoch:  389	assembling the matrix time taken:  0.00023603439331054688
solving Ax = b time taken:  0.02834463119506836
epoch:  390	assembling the matrix time taken:  0.00024318695068359375
solving Ax = b time taken:  0.029264450073242188
epoch:  391	assembling the matrix time taken:  0.00023555755615234375
solving Ax = b time taken:  0.028472900390625
epoch:  392	assembling the matrix time taken:  0.00024175643920898438
solving Ax = b time taken:  0.029083251953125
epoch:  393	assembling the matrix time taken:  0.0002372264862060547
solving Ax = b time taken:  0.028682470321655273
epoch:  394	

solving Ax = b time taken:  0.0411076545715332
epoch:  458	assembling the matrix time taken:  0.0002474784851074219
solving Ax = b time taken:  0.03858828544616699
epoch:  459	assembling the matrix time taken:  0.00023794174194335938
solving Ax = b time taken:  0.04183840751647949
epoch:  460	assembling the matrix time taken:  0.0002551078796386719
solving Ax = b time taken:  0.040120601654052734
epoch:  461	assembling the matrix time taken:  0.00023674964904785156
solving Ax = b time taken:  0.037163496017456055
epoch:  462	assembling the matrix time taken:  0.00025081634521484375
solving Ax = b time taken:  0.03615975379943848
epoch:  463	assembling the matrix time taken:  0.0002357959747314453
solving Ax = b time taken:  0.03701949119567871
epoch:  464	assembling the matrix time taken:  0.0002455711364746094
solving Ax = b time taken:  0.035570383071899414
epoch:  465	assembling the matrix time taken:  0.00023627281188964844
solving Ax = b time taken:  0.03721952438354492
epoch:  46